In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.utils

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

22/03/07 14:25:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Q1

In [3]:
spark.version

'3.0.3'

In [3]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-07 14:22:33--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.138.227
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.138.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  12.0MB/s    in 70s     

2022-03-07 14:23:44 (10.1 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [4]:
df = spark.read \
    .option("header","true") \
    .csv("fhvhv_tripdata_2021-02.csv")

df.repartition(24).write.parquet('fhvhv_tripdata/')

### Q2

In [1]:
!ls -lh fhvhv_tripdata_2021-02.csv

-rw-rw-r-- 1 mufidnuha mufidnuha 700M Oct 29 18:53 fhvhv_tripdata_2021-02.csv


In [5]:
!ls -lh fhvhv_tripdata/

total 208M
-rw-r--r-- 1 mufidnuha mufidnuha    0 Mar  7 14:27 _SUCCESS
-rw-r--r-- 1 mufidnuha mufidnuha 8.7M Mar  7 14:26 part-00000-9525d93c-9706-48c3-b8db-3194e1ac4612-c000.snappy.parquet
-rw-r--r-- 1 mufidnuha mufidnuha 8.7M Mar  7 14:26 part-00001-9525d93c-9706-48c3-b8db-3194e1ac4612-c000.snappy.parquet
-rw-r--r-- 1 mufidnuha mufidnuha 8.7M Mar  7 14:26 part-00002-9525d93c-9706-48c3-b8db-3194e1ac4612-c000.snappy.parquet
-rw-r--r-- 1 mufidnuha mufidnuha 8.7M Mar  7 14:26 part-00003-9525d93c-9706-48c3-b8db-3194e1ac4612-c000.snappy.parquet
-rw-r--r-- 1 mufidnuha mufidnuha 8.7M Mar  7 14:26 part-00004-9525d93c-9706-48c3-b8db-3194e1ac4612-c000.snappy.parquet
-rw-r--r-- 1 mufidnuha mufidnuha 8.7M Mar  7 14:26 part-00005-9525d93c-9706-48c3-b8db-3194e1ac4612-c000.snappy.parquet
-rw-r--r-- 1 mufidnuha mufidnuha 8.7M Mar  7 14:26 part-00006-9525d93c-9706-48c3-b8db-3194e1ac4612-c000.snappy.parquet
-rw-r--r-- 1 mufidnuha mufidnuha 8.7M Mar  7 14:26 part-00007-9525d93c-9706-48c3-b8db-3194e1ac46

In [6]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [7]:
df.registerTempTable('trip_data')

### Q3

In [21]:
df_results = spark.sql("""
SELECT COUNT(*)
FROM trip_data
WHERE CAST(pickup_datetime AS DATE) = '2021-02-15'
""")

In [22]:
df_results.show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



### Q4

In [114]:
duration_trip = spark.sql("""
SELECT 
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime))/3600 AS duration,
    CAST(pickup_datetime AS DATE) AS day
FROM trip_data
""")

In [115]:
duration_trip.registerTempTable('duration_trip')
longest_trip = spark.sql("""
SELECT
    day
FROM
    duration_trip
WHERE
    duration = (SELECT MAX(duration) FROM duration_trip)
""")

In [116]:
longest_trip.show()

+----------+
|       day|
+----------+
|2021-02-11|
+----------+



### Q5

In [117]:
df.registerTempTable('trip_data')

In [118]:
freq_db = spark.sql("""
SELECT 
    dispatching_base_num, COUNT(dispatching_base_num) AS num_db
FROM
    trip_data
GROUP BY dispatching_base_num
""")

In [119]:
freq_db.registerTempTable('freq_db')
most_db = spark.sql("""
SELECT
    dispatching_base_num
FROM
    freq_db
WHERE
    num_db = (SELECT MAX(num_db) FROM freq_db)
""")

In [120]:
most_db.show()

+--------------------+
|dispatching_base_num|
+--------------------+
|              B02510|
+--------------------+



In [ ]:
#answer: 3 stages

### Q6

In [87]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-03-07 15:21:49--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.169.168
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.169.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-03-07 15:21:50 (88.7 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [88]:
taxi_zone = spark.read \
            .option("header","true") \
            .csv('taxi+_zone_lookup.csv')

In [90]:
df.registerTempTable('trip_data')
taxi_zone.registerTempTable('taxi_zone')

In [106]:
loc_pair = spark.sql("""
SELECT
    PULocationID, tz.Zone, DOLocationID, tz2.Zone, CONCAT(tz.Zone, ' / ', tz2.Zone) AS zone_pair
FROM
    trip_data AS td
        LEFT JOIN taxi_zone AS tz
        ON td.PULocationID = tz.LocationID
        LEFT JOIN taxi_zone AS tz2
        ON td.DOLocationID = tz2.LocationID
""")

In [108]:
loc_pair.registerTempTable('loc_pair')

In [111]:
most_pair = spark.sql("""
SELECT
    zone_pair, COUNT(zone_pair) AS num_pair
FROM
    loc_pair
GROUP BY zone_pair
""")

In [112]:
most_pair.show()

+--------------------+--------+
|           zone_pair|num_pair|
+--------------------+--------+
|  Midtown South / NA|    3380|
|Hamilton Heights ...|    1452|
|Hamilton Heights ...|     960|
|Bedford Park / Be...|    5063|
|Bushwick South / ...|     372|
|    Inwood / Norwood|     664|
|East Flatbush/Rem...|    5659|
|Alphabet City / C...|     430|
|Park Slope / Park...|    9771|
|East Harlem South...|     107|
|Homecrest / Grave...|    1522|
|Eastchester / Hun...|     338|
|TriBeCa/Civic Cen...|     566|
|Norwood / Kensington|       4|
|Greenwich Village...|     675|
|Upper West Side N...|    1415|
|Kew Gardens Hills...|     157|
|Long Island City/...|    1487|
|Brownsville / Two...|     160|
|Soundview/Castle ...|      64|
+--------------------+--------+
only showing top 20 rows



In [113]:
most_pair.registerTempTable('most_pair')

In [121]:
most_common = spark.sql("""
SELECT
    zone_pair
FROM
    most_pair
WHERE
    num_pair = (SELECT MAX(num_pair) FROM most_pair)
"""
)

In [122]:
most_common.show()

+--------------------+
|           zone_pair|
+--------------------+
|East New York / E...|
+--------------------+

